In [1]:
%matplotlib widget

In [2]:
import flammkuchen as fl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json
import imageio
from scipy.stats import sem
import seaborn as sns
from bouter.free import FreelySwimmingExperiment
import pandas as pd
from scipy import signal
from tqdm import tqdm
from scipy.stats import zscore
from scipy.interpolate import interp1d
from sklearn.neighbors import KernelDensity as kde
from scipy.stats import linregress
from scipy.signal import medfilt
from scipy.signal import savgol_filter
from scipy.stats import mannwhitneyu

In [3]:
def clip(x_):
    x_[x_>200]=200
    x_[x_<0]= 0
    return x_
def clean_augment(x_,stim_fun,setpoint):
    out_ = []
    for df_ in x_:
        if "flip_x_start" in df_.keys():
            pass
        else:
            df_["flip_x_start"] = df_["al_start_x"]
            df_["flip_y_start"] = df_["al_start_y"]
            
            df_["flip_x_end"] = df_["al_end_x"]
            df_["flip_y_end"] = df_["al_end_y"]
        df_["flip_x_start"] = clip(df_["flip_x_start"].values)
        try:
            df_["flip_x_end"] = clip(df_["flip_x_end"].values)
        except:
            df_["flip_x_end"] = clip(df_["al_end_x"].values)
        out_.append(augment(df_,stim_fun,setpoint))
    return out_


def dist_setpoint(values,setpoint):
    return abs(values - setpoint)

def augment(bout_properties,fun,setpoint,time_th=2):
    border_x = [10, 190]
    border_y = [5, 45]
    x_start = []
    x_end = []
    temp_start = []
    temp_end = []
    temp_pre = []
    delta = []
    delta_prev = []
    out = []
    prev_biases = []
    temporal_distance = []
    prev_bout = None
    prev_bias = None
    prev_temp = None
    delta_prev_prev_bout = None
    delta_prev_bout = np.nan
    for i in range(len(bout_properties)):
        
        bout = bout_properties.iloc[i]
        if prev_bout is not None:
            temporal_distance = bout["t_start"] - prev_bout["t_end"] 
            if (temporal_distance <= time_th)& (delta_prev_bout!=np.nan):
                delta_prev_prev_bout = delta_prev_bout
            else:
                delta_prev_prev_bout  = np.nan
                
        if prev_bout is not None:
            temporal_distance = bout["t_start"] - prev_bout["t_end"] 
            if temporal_distance <= time_th:
                prev_bias = prev_bout["theta_bout"]
                
                delta_prev_bout = dist_setpoint(float(fun(prev_bout["flip_x_start"])),setpoint) - dist_setpoint(float(fun(bout["flip_x_start"])),setpoint)
                prev_temp = dist_setpoint(float(fun(prev_bout["flip_x_start"])),setpoint)
            else:
                delta_prev_bout  = np.nan
                prev_bias = np.nan
                prev_temp = np.nan
            
        temp_start.append(dist_setpoint(float(fun(bout["flip_x_start"])),setpoint))
        delta.append(delta_prev_bout)
        delta_prev.append(delta_prev_prev_bout)
        x_start.append(bout['flip_x_start'])
        temp_pre.append(prev_temp)
        prev_biases.append(prev_bias)
        if (border_x[0] <= bout["flip_x_start"] <= border_x[1]) and (border_y[0] <= bout["y_start"] <= border_y[1]):
            is_out = False
        else:
            is_out = True
        out.append(is_out)
        prev_bout = bout.copy()
    bout_properties["temp_start"] = temp_start
    bout_properties["delta"]=delta
    bout_properties["delta_1"]=delta_prev
    bout_properties["out"]=out
    bout_properties["prev_bias"]=prev_biases
    bout_properties["prev_temp"] = temp_pre
    return bout_properties

def prefered_temperature(x_position,times,resolution=2):
    result = medfilt(np.interp(np.linspace(0,15*60,15*60*resolution),
            times,x_position),3)
    return np.nanmedian(result[600*resolution:])

def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

In [4]:
plt.rcParams.update({'font.size': 25})

In [5]:
setpoint_wt = 25.280104571722397
setpoint_hb = 27.89187878451635

In [7]:
full_dataset = fl.load(r"F:\experiments\virtual_gradients_experiments\freely_swim_Ruben\data.h5")
transfer_fun = interp1d(full_dataset["stimulus_calibration"][0,:],
                             full_dataset["stimulus_calibration"][1,:],
                             kind="quadratic")

WT = clean_augment(full_dataset["WT"].copy(),transfer_fun,setpoint_wt)
Hbpos_pre = clean_augment(full_dataset["Hbpos_pre_ablation"].copy(),transfer_fun,setpoint_hb)
Hbneg_post = clean_augment(full_dataset["Hbneg_post_ablation"].copy(),transfer_fun,setpoint_hb)
Hbpos_post = clean_augment(full_dataset["Hbpos_ablation"].copy(),transfer_fun,setpoint_hb)

In [8]:
print(len(Hbpos_pre),len(Hbneg_post),len(Hbpos_post))

25 23 14


In [8]:
def square(bins_,to_split_x,to_split_y,to_quant,add=None):
    out_ = np.zeros((bins_.shape[0]-1,bins_.shape[0]-1))*np.nan
    counts_ = np.zeros((bins_.shape[0]-1,bins_.shape[0]-1))*np.nan
    for p0,(s_0,e_0) in enumerate(zip(bins_[:-1],bins_[1:])):
        for p1,(s_1,e_1) in enumerate(zip(bins_[:-1],bins_[1:])):
            to_take = (to_split_x>=s_0)&(to_split_y>=s_1)&(to_split_x<e_0)&(to_split_y<e_1)
            selected = to_quant[to_take]
            #value = np.nansum(abs(selected)>np.deg2rad(10)) / selected.shape[0]
            value = np.nanmedian(selected)#*np.sign(add[to_take])))
            out_[p1,p0] = value
            counts_[p1,p0] = selected.shape[0]
    return out_*(counts_>100)

In [13]:
dist_pt_th = -1
threshold_dpt = 0.1333
mod_index_wt_up_prev = []
mod_index_wt_down_prev = []

mod_index_wt_up_current = []
mod_index_wt_down_current = []

for i_fish, fish_ in enumerate(WT):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    in_iso = (dist_pt>-threshold_dpt)&(dist_pt<threshold_dpt)
    iso_bouts = fish_.iloc[is_in].iloc[in_iso]
    above_th = (iso_bouts["delta_1"].values<-threshold_dpt)#&(abs(iso_bouts["prev_bias"].values)>np.deg2rad(30))
    
    choice_bout = iso_bouts["theta_bout"].values[above_th]
 
    factor = np.sign(iso_bouts["prev_bias"].values[above_th])
    mod_index_wt_up_prev.extend(iso_bouts["prev_bias"].values[above_th]*factor)
    mod_index_wt_up_current.extend(iso_bouts["theta_bout"].values[above_th]*factor)
    
    below_th = (iso_bouts["delta_1"].values>threshold_dpt)#&(abs(iso_bouts["prev_bias"].values)>np.deg2rad(30))
    choice_bout = iso_bouts["theta_bout"].values[below_th]
    #choice_bout[iso_bouts["prev_bias"].values[below_th]<0] = -choice_bout[iso_bouts["prev_bias"].values[below_th]<0]
    #c_down,_ = np.histogram(choice_bout, np.array([-np.inf,-np.deg2rad(10),np.deg2rad(10),np.inf]))
    #c_down = c_down / np.sum(c_down)
    factor = np.sign(iso_bouts["prev_bias"].values[below_th])
    mod_index_wt_down_prev.extend(iso_bouts["prev_bias"].values[below_th]*factor)
    mod_index_wt_down_current.extend(iso_bouts["theta_bout"].values[below_th]*factor)
#mod_index_wt_down = np.stack(mod_index_wt_down,0)

C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in greater
  if sys.path[0] == '':
C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in less
  if sys.path[0] == '':
C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  
C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater


In [442]:
dist_pt_th = -1
threshold_dpt = 0.05
mod_index_wt = []
for i_fish, fish_ in enumerate(WT):
    is_in = fish_["out"].values == False
    distant = fish_["temp_start"].values >= dist_pt_th
    dist_pt = fish_["delta"].values[is_in&distant]
    in_iso = (dist_pt>-threshold_dpt)&(dist_pt<threshold_dpt)
    iso_bouts = fish_.iloc[is_in&distant].iloc[in_iso]
    above_th = iso_bouts["delta_1"].values<-threshold_dpt
    #below_th = iso_bouts["delta_1"].values<-threshold_dpt
    choice_bout = iso_bouts["theta_bout"].values[above_th]
    choice_bout[iso_bouts["prev_bias"].values[above_th]<0] = -choice_bout[iso_bouts["prev_bias"].values[above_th]<0]
    mod_index_wt.append(li(choice_bout))
mod_index_wt = np.stack(mod_index_wt,0)


mod_index_Hbneg_post = []
for i_fish, fish_ in enumerate(Hbneg_post):
    is_in = fish_["out"].values == False
    distant = fish_["temp_start"].values >= dist_pt_th
    dist_pt = fish_["delta"].values[is_in&distant]
    in_iso = (dist_pt>-threshold_dpt)&(dist_pt<threshold_dpt)
    iso_bouts = fish_.iloc[is_in&distant].iloc[in_iso]
    above_th = iso_bouts["delta_1"].values<-threshold_dpt
    #below_th = iso_bouts["delta_1"].values<-threshold_dpt
    choice_bout = iso_bouts["theta_bout"].values[above_th]
    choice_bout[iso_bouts["prev_bias"].values[above_th]<0] = -choice_bout[iso_bouts["prev_bias"].values[above_th]<0]
    mod_index_Hbneg_post.append(li(choice_bout))
mod_index_Hbneg_post = np.stack(mod_index_Hbneg_post,0)

mod_index_Hbpos_pre = []
for i_fish, fish_ in enumerate(Hbpos_pre):
    is_in = fish_["out"].values == False
    distant = fish_["temp_start"].values >= dist_pt_th
    dist_pt = fish_["delta"].values[is_in&distant]
    in_iso = (dist_pt>-threshold_dpt)&(dist_pt<threshold_dpt)
    iso_bouts = fish_.iloc[is_in&distant].iloc[in_iso]
    above_th = iso_bouts["delta_1"].values<-threshold_dpt
    #below_th = iso_bouts["delta_1"].values<-threshold_dpt
    choice_bout = iso_bouts["theta_bout"].values[above_th]
    choice_bout[iso_bouts["prev_bias"].values[above_th]<0] = -choice_bout[iso_bouts["prev_bias"].values[above_th]<0]
    mod_index_Hbpos_pre.append(li(choice_bout))
mod_index_Hbpos_pre = np.stack(mod_index_Hbpos_pre,0)

mod_index_Hbpos_post = []
for i_fish, fish_ in enumerate(Hbpos_post):
    is_in = fish_["out"].values == False
    distant = fish_["temp_start"].values >= dist_pt_th
    dist_pt = fish_["delta"].values[is_in&distant]
    in_iso = (dist_pt>-threshold_dpt)&(dist_pt<threshold_dpt)
    iso_bouts = fish_.iloc[is_in&distant].iloc[in_iso]
    above_th = iso_bouts["delta_1"].values<-threshold_dpt
    #below_th = iso_bouts["delta_1"].values<-threshold_dpt
    choice_bout = iso_bouts["theta_bout"].values[above_th]
    choice_bout[iso_bouts["prev_bias"].values[above_th]<0] = -choice_bout[iso_bouts["prev_bias"].values[above_th]<0]
    mod_index_Hbpos_post.append(li(choice_bout))
mod_index_Hbpos_post = np.stack(mod_index_Hbpos_post,0)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


In [443]:
plt.figure()
to_plot = mod_index_Hbneg_post.copy()
x_points = np.random.uniform(low=-1,high=1,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.3,c="orange",linewidths=0)
plt.errorbar([0],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="orange",
            capthick=5)

to_plot = mod_index_Hbpos_pre.copy()
x_points = np.random.uniform(low=-4,high=-2,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.3,c="orange",linewidths=0)
plt.errorbar([-3],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="orange",
            capthick=5)

to_plot = mod_index_Hbpos_post.copy()
x_points = np.random.uniform(low=2,high=4,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.3,c="orange",linewidths=0)
plt.errorbar([3],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="orange",
            capthick=5)
plt.plot([-6,6],[0,0],"--k",alpha=.5)
plt.ylim([-1,1])
plt.xlim([-6,6])
plt.xticks([])
plt.yticks([-1,-.5,0,0.5,1])
plt.ylabel("Modulation Index",fontsize=20)
sns.despine(bottom=True)
plt.tight_layout()
plt.show()

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
print(mannwhitneyu(mod_index_Hbpos_post,mod_index_Hbneg_post))
print(mannwhitneyu(mod_index_Hbpos_post,mod_index_Hbpos_pre))

MannwhitneyuResult(statistic=145.0, pvalue=0.20964426626215754)
MannwhitneyuResult(statistic=165.0, pvalue=0.2691266806353121)


In [445]:
plt.figure()
to_plot = mod_index_wt.copy()
x_points = np.random.uniform(low=-1,high=1,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.2,c="k",linewidths=0)
plt.errorbar([0],y=np.array([np.nanmean(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="k",
            capthick=5)

plt.plot([-6,6],[0,0],"--k",alpha=.5)
plt.ylim([-.4,.4])
plt.xlim([-6,6])
plt.xticks([])
plt.yticks([-1,-.5,0,0.5,1])
plt.ylabel("Modulation Index",fontsize=20)
sns.despine(bottom=True)
plt.tight_layout()
plt.show()

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
print(mannwhitneyu(mod_index_wt,mod_index_fluo))
print(mannwhitneyu(mod_index_wt,mod_index_cpca))

MannwhitneyuResult(statistic=335.0, pvalue=0.10408160744205913)
MannwhitneyuResult(statistic=231.0, pvalue=0.05900507618850612)


In [52]:
plt.figure()
plt.scatter([-1.7893538198068981,
 -6.91711958334529,
 -7.093098760688196,
 -0.5580348065612704,
 0.1795763114405453,
 1.3120250397540474,
 2.5685949717730203,
 -2.624666933582141,
 -0.8662748830788622,
 -7.151430523506239,
 -2.774899695215172,
 -2.4577400934316422,
 -4.9519634463686835,
 -1.1561146337181825,
 -0.7544948123262145,
 -3.1811251935966887,
 -1.1477060733774636,
 -1.5421942568483367,
 -0.4699990993332328,
 -0.16951450526278933,
 -0.8151721503708806,
 -1.763795857548427,
 -0.07630000499121081,
 3.2192133962704936,
 -0.969438620446919,
 -0.13478318780575194],mod_index_wt)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
threshold_dpt = 0.1333
mod_index_wt = []
for i_fish, fish_ in enumerate(WT):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    iso_bouts = fish_.iloc[is_in]
    above_th = iso_bouts.iloc[iso_bouts["delta"].values<-threshold_dpt]
    below_th = iso_bouts.iloc[iso_bouts["delta"].values>threshold_dpt]
    iso_th = iso_bouts.iloc[abs(iso_bouts["delta"].values)<=threshold_dpt]
    above_th["theta_bout"][above_th["prev_bias"].values<0] = -above_th["theta_bout"][above_th["prev_bias"].values<0]
    below_th["theta_bout"][below_th["prev_bias"].values<0] = -below_th["theta_bout"][below_th["prev_bias"].values<0]
    iso_th["theta_bout"][iso_th["prev_bias"].values<0] = -iso_th["theta_bout"][iso_th["prev_bias"].values<0]
#     were_turning_below = abs(below_th["prev_bias"].values) >=np.deg2rad(20)
#     were_turning_above = abs(above_th["prev_bias"].values) >=np.deg2rad(20)
    mod_index_wt.append(np.array([pipsi(below_th["theta_bout"].values)-
                                  pipsi(above_th["theta_bout"].values)]))
mod_index_wt = np.stack(mod_index_wt,0)

mod_index_fluo = []
for i_fish, fish_ in enumerate(fluo):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    iso_bouts = fish_.iloc[is_in]
    above_th = iso_bouts.iloc[iso_bouts["delta"].values<-threshold_dpt]
    below_th = iso_bouts.iloc[iso_bouts["delta"].values>threshold_dpt]
    iso_th = iso_bouts.iloc[abs(iso_bouts["delta"].values)<=threshold_dpt]
    above_th["theta_bout"][above_th["prev_bias"].values<0] = -above_th["theta_bout"][above_th["prev_bias"].values<0]
    below_th["theta_bout"][below_th["prev_bias"].values<0] = -below_th["theta_bout"][below_th["prev_bias"].values<0]
    iso_th["theta_bout"][iso_th["prev_bias"].values<0] = -iso_th["theta_bout"][iso_th["prev_bias"].values<0]

    mod_index_fluo.append(np.array([pipsi(below_th["theta_bout"].values)-
                                  pipsi(above_th["theta_bout"].values)]))
mod_index_fluo = np.stack(mod_index_fluo,0)

mod_index_cpca = []
for i_fish, fish_ in enumerate(cpca):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    iso_bouts = fish_.iloc[is_in]
    above_th = iso_bouts.iloc[iso_bouts["delta"].values<-threshold_dpt]
    below_th = iso_bouts.iloc[iso_bouts["delta"].values>threshold_dpt]
    iso_th = iso_bouts.iloc[abs(iso_bouts["delta"].values)<=threshold_dpt]
    above_th["theta_bout"][above_th["prev_bias"].values<0] = -above_th["theta_bout"][above_th["prev_bias"].values<0]
    below_th["theta_bout"][below_th["prev_bias"].values<0] = -below_th["theta_bout"][below_th["prev_bias"].values<0]
    iso_th["theta_bout"][iso_th["prev_bias"].values<0] = -iso_th["theta_bout"][iso_th["prev_bias"].values<0]

    mod_index_cpca.append(np.array([pipsi(below_th["theta_bout"].values)-
                                  pipsi(above_th["theta_bout"].values)]))
mod_index_cpca = np.stack(mod_index_cpca,0)

mod_index_Hbneg_post = []
for i_fish, fish_ in enumerate(Hbneg_post):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    iso_bouts = fish_.iloc[is_in]
    above_th = iso_bouts.iloc[iso_bouts["delta"].values<-threshold_dpt]
    below_th = iso_bouts.iloc[iso_bouts["delta"].values>threshold_dpt]
    iso_th = iso_bouts.iloc[abs(iso_bouts["delta"].values)<=threshold_dpt]
    above_th["theta_bout"][above_th["prev_bias"].values<0] = -above_th["theta_bout"][above_th["prev_bias"].values<0]
    below_th["theta_bout"][below_th["prev_bias"].values<0] = -below_th["theta_bout"][below_th["prev_bias"].values<0]
    iso_th["theta_bout"][iso_th["prev_bias"].values<0] = -iso_th["theta_bout"][iso_th["prev_bias"].values<0]

    mod_index_Hbneg_post.append(np.array([pipsi(below_th["theta_bout"].values)-
                                  pipsi(above_th["theta_bout"].values)]))
mod_index_Hbneg_post = np.stack(mod_index_Hbneg_post,0)

mod_index_Hbpos_pre = []
for i_fish, fish_ in enumerate(Hbpos_pre):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    iso_bouts = fish_.iloc[is_in]
    above_th = iso_bouts.iloc[iso_bouts["delta"].values<-threshold_dpt]
    below_th = iso_bouts.iloc[iso_bouts["delta"].values>threshold_dpt]
    iso_th = iso_bouts.iloc[abs(iso_bouts["delta"].values)<=threshold_dpt]
    above_th["theta_bout"][above_th["prev_bias"].values<0] = -above_th["theta_bout"][above_th["prev_bias"].values<0]
    below_th["theta_bout"][below_th["prev_bias"].values<0] = -below_th["theta_bout"][below_th["prev_bias"].values<0]
    iso_th["theta_bout"][iso_th["prev_bias"].values<0] = -iso_th["theta_bout"][iso_th["prev_bias"].values<0]

    mod_index_Hbpos_pre.append(np.array([pipsi(below_th["theta_bout"].values)-
                                  pipsi(above_th["theta_bout"].values)]))
mod_index_Hbpos_pre = np.stack(mod_index_Hbpos_pre,0)

mod_index_Hbpos_post = []
for i_fish, fish_ in enumerate(Hbpos_post):
    is_in = fish_["out"].values == False
    dist_pt = fish_["delta"].values[is_in]
    iso_bouts = fish_.iloc[is_in]
    above_th = iso_bouts.iloc[iso_bouts["delta"].values<-threshold_dpt]
    below_th = iso_bouts.iloc[iso_bouts["delta"].values>threshold_dpt]
    iso_th = iso_bouts.iloc[abs(iso_bouts["delta"].values)<=threshold_dpt]
    above_th["theta_bout"][above_th["prev_bias"].values<0] = -above_th["theta_bout"][above_th["prev_bias"].values<0]
    below_th["theta_bout"][below_th["prev_bias"].values<0] = -below_th["theta_bout"][below_th["prev_bias"].values<0]
    iso_th["theta_bout"][iso_th["prev_bias"].values<0] = -iso_th["theta_bout"][iso_th["prev_bias"].values<0]

    mod_index_Hbpos_post.append(np.array([pipsi(below_th["theta_bout"].values)-
                                  pipsi(above_th["theta_bout"].values)]))
mod_index_Hbpos_post = np.stack(mod_index_Hbpos_post,0)

C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys
C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in less_equal
  if __name__ == '__main__':
C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\epaoli\anaconda3\lib\site-packages\pandas\core\generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-

C:\Users\epaoli\anaconda3\lib\site-packages\ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
plt.figure()
to_plot = mod_index_wt.copy()[:,0]
x_points = np.random.uniform(low=-1,high=1,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.2,c="k",linewidths=0)
plt.errorbar([0],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="k",
            capthick=5)

to_plot = mod_index_cpca.copy()[:,0]
x_points = np.random.uniform(low=-4,high=-2,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.2,c="k",linewidths=0)
plt.errorbar([-3],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="k",
            capthick=5)

to_plot = mod_index_fluo.copy()[:,0]
x_points = np.random.uniform(low=2,high=4,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.2,c="k",linewidths=0)
plt.errorbar([3],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="k",
            capthick=5)
plt.plot([-6,6],[0,0],"--k",alpha=.5)
plt.ylim([-1,1])
plt.xlim([-6,6])
plt.xticks([])
plt.yticks([-1,-.5,0,0.5,1])
plt.ylabel("Modulation Index",fontsize=20)
sns.despine(bottom=True)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.figure()
to_plot = mod_index_Hbneg_post.copy()[:,0]
x_points = np.random.uniform(low=-1,high=1,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.3,c="orange",linewidths=0)
plt.errorbar([0],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="orange",
            capthick=5)

to_plot = mod_index_Hbpos_pre.copy()[:,0]
x_points = np.random.uniform(low=-4,high=-2,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.3,c="orange",linewidths=0)
plt.errorbar([-3],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="orange",
            capthick=5)

to_plot = mod_index_Hbpos_post.copy()[:,0]
x_points = np.random.uniform(low=2,high=4,size=(len(to_plot)))
plt.scatter(x_points,to_plot,s=50,alpha=.3,c="orange",linewidths=0)
plt.errorbar([3],y=np.array([np.nanmedian(to_plot)]),
            yerr=sem(to_plot,nan_policy="omit")*1.2,
            lw=4,fmt="-",ms=20,marker="_",mew=2,c="orange",
            capthick=5)
plt.plot([-6,6],[0,0],"--k",alpha=.5)
plt.ylim([-1,1])
plt.xlim([-6,6])
plt.xticks([])
plt.yticks([-1,-.5,0,0.5,1])
plt.ylabel("Modulation Index",fontsize=20)
sns.despine(bottom=True)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …